## Part 1: NYC taxi data

### Basic Questions

1. What is the distribution of number of passengers per trip? (done)
2. What is the distribution of payment_type? (fares)
3. What is the distribution of fare amount? (fares)
4. What is the distribution of tip amount? (fares)
5. What is the distribution of total amount? (fares)
6. What are top 5 busiest hours of the day? (trips)
7. What are the top 10 busiest locations of the city? (trips)
8. Which trip has the highest standard deviation of travel time? (trips)
9. Which trip has most consistent fares? (fares)

### Open Questions

1. In what trips can you confidently use respective means as measures of central tendency
to estimate fare, time taken, etc.

    - Suppose it's about the fare. When we say "confidently use" we mean that knowing the mean fare should give us a resonably accurate sense of what this fare should be typically. Knowing what fare to expect helps make "right" decisions such as "ok, I'm happy to pay about 10 usd to get to Brooklyn". Or maybe "it's 10 bucks which i think is too much, i'm catching metro for 2.50". We'd use the means for the trips which cost roughly the same over the recorded 30 days, i.e. there were no clearly unusual charges ("outliers"). The reason is that outliers can strongly affect means. For example, let's say the fares for a trip recorded 10 times were about 1 dollar 9 times and then suddenly 20 dollars. The mean would be (9*1+20)/10=2.90 which is almost triple the typical ~1 dollar fare. Instead of means we'd use measures unaffected by extreme values, e.g. median which would be 1 dollar in this example - clearly, a much more accurate estimation.
    
2. Can we build a model to predict fare and tip amount given pick up and drop off
coordinates, time of day and week?

    - Of course. One of the simplest and yet completely usable models would be the one showing you some of measures mentioned in question 1. For example, we can put together a model that (based on the data we have) calculates median tip and mean fare for any  pair of the pickup and dropoff locations by time of day and week. Should these measures be accurate enough estimates (as discussed in question 1), we have a cheap and accurate for our purposes model on our hands. What if some pair of locations is not in the data? Then we introduce a straightforward extension: the model will now find the nearest locations for which the fare and tip amounts are avaiable, calculate the measures and produce these as the best estimates. To take this a notch higher, we can build a regression model that takes locations, time of day and week as variables.
    
3. If you were a taxi owner, how would you maximize your earnings in a day?

    - While there would be multiple strategies to do this, we could stick with a "greedy" approach, i.e. maximising the earnings as we go. That is, once a trip is completed all we think about is how to make the next trip most profitable and don't care about more complex strategies that may involve giving up lucrative rides one time only to earn more another time. A straightforward approach to earning as much as possible would be then to do as many rides as possible. Which in turn would suggest heading to the nearest area where the highest demand is at a particular time. On a higher level, this can be framed as an optimisation problem when there's a function (we can call it a "profit function" in this case) describing earnings depending on the things  (variables) perceived important. For example, distance to the pickup spot, likely gas spend, trip distance, etc. We then figure out the values of these variables which will make the function spit out the highest value (profit).
    
4. If you were a taxi owner, how would you minimize your work time while retaining the
average wages earned by a typical taxi in the dataset?

    - This is a setup for another optimisation problem similar to the one mentioned in the previous question. The difference would be in the function we would care about. Specifically, the function would produce the work time depending on the factors potentially affecting this time (all of the available data points may have an effect, direct or indirect). Our goal would be to minimise the value the function produces. However, we also want to make sure the average wages are not getting worse. To ensure that, we put together another function that produces average wages when given all important inputs the wages are likely to depend on. This function must return either exactly the existing average wages (if we decide to stick to retain, literally) or higher wages which we might like even better. The former leads to a problem of minimising the work time under an equality constrain (we will only accept the inputs making the time function minimal if these don't stand in the way of the average wages function producing the exact same amount as now) while the latter involves an inequality constrain (the average wages need to be >= the current ones).
    
    
5. If you run a taxi company with 10 taxis, how would you maximize your earnings?

    - Having 10 taxis changes the perspective a bit. Now we can pay more attention to positioning each car in an optimal way which is not necessarily to be in the right time and place to get the biggest profit per car, especially if we start accounting for the competitors activities (what if some of our cars are also contributing to creating additional profit opportunities by interfering with the competitots plans?). 
    
[Optimization of the Revenue of the New York City Taxi Service
using Markov Decision Processes]

## Part 2: Open showcase
After the presentation of you results for “NYC taxi data” case, you’ll have the opportunity to present your
own work for 15 minutes.
Please illustrate with an achievement of your choice (e.g. publication, participation in the community,
portfolio, open repository, etc).

### Available Data

**Note**: many column names begin with a white space!

The **trip**-related data points are the following:

* 'medallion',
* ' hack_license',
* ' vendor_id',
* ' rate_code',
* ' store_and_fwd_flag',
* ' pickup_datetime',
* ' dropoff_datetime',
* ' passenger_count',
* ' trip_time_in_secs',
* ' trip_distance',
* ' pickup_longitude',
* ' pickup_latitude',
* ' dropoff_longitude',
* ' dropoff_latitude'

and the **fare** CSV file contains the columns as below

* 'medallion',
* ' hack_license',
* ' vendor_id',
* ' pickup_datetime',
* ' payment_type',
* ' fare_amount',
* ' surcharge',
* ' mta_tax',
* ' tip_amount',
* ' tolls_amount',
* ' total_amount'

In [1]:
import pandas as pd
import re
import seaborn as sns
import arrow
import time
from collections import Counter, defaultdict, namedtuple
import geopandas as gpd
import numpy as np
from geopy.geocoders import Nominatim

sns.set_style("whitegrid")

nom = Nominatim(user_agent='app', timeout=10)

In [2]:
class Taxi:
    
    def __init__(self, trip_file, fare_file):
        
        Location = namedtuple('Location', 'lat lng')
        
        self.EMP_STATE_BUILDING = Location(lat=40.7484, lng=-73.9856)
        
        self.extreme_trip_index = None
        self.popular_locations = None
        self.AV_TRIP_TIME = None
        self.STD_TRIP_TIME = None
        
        self.ANSWERS = defaultdict()
        
        self.FILES = {'trips': 'data/' + trip_file,
                     'fares': 'data/' + fare_file}
        self.COLS = {'latitude': [' pickup_latitude', ' dropoff_latitude'],
                     'longitude': [' pickup_longitude', ' dropoff_longitude'],
                     'useless': [' vendor_id', ' rate_code', ' store_and_fwd_flag', 
                                 ' hack_license', ' vendor_id', ' tolls_amount',
                                ' mta_tax', ' surcharge', ' tip_amount'],
                     'amount': [' fare_amount', ' tip_amount', ' total_amount']}
        
        self.distros = defaultdict(lambda: defaultdict(int))
        
    def fix_coords(self, c, lat_lng):
        
        """
        returns the original longitude or latitude rounded to 3 decimal digits if their values are not too far
        from the Empire State Building; otherwise returns None
        
        we are being very generous here weeding out only the most remote locations (0 decimal places 
        i.e. something like 100km away) - 
        see https://en.wikipedia.org/wiki/Decimal_degrees
        """
        
        c_ = 'lng' if lat_lng == 'longitude' else 'lat' if lat_lng == 'latitude' else None
        
        if not c_:
            raise Exception(f'{self.fix_coords.__name__}: argument lat_lng is neither latitude nor longitude!')
            
        try:
            return round(float(c),3) if abs(float(c) - getattr(self.EMP_STATE_BUILDING, c_)) < 1 else None
        except:
            pass
        
    def get_and_process_data(self, show_counts=False, sample_n=None):
        
        self.trips = pd.read_csv(self.FILES['trips'], dtype=str)
        self.trips = self.trips.drop([c for c in self.trips.columns if c in self.COLS['useless']], axis=1)
        
        if sample_n:
            self.trips = self.trips.sample(n=sample_n)
        
        self.fares = pd.read_csv(self.FILES['fares'], dtype=str)
        self.fares = self.fares.drop([c for c in self.fares.columns if c in self.COLS['useless']], axis=1)
        
        if sample_n:
            self.fares = self.fares[self.fares['medallion'].isin(self.trips['medallion']) & 
                                   self.fares[' pickup_datetime'].isin(self.trips[' pickup_datetime'])]
        
        if show_counts:
            
            print(f'cabs: {self.trips["medallion"].nunique():,}')
            print(f'drivers: {self.trips[" hack_license"].nunique():,}')
            # note that the date format is 2013-04-04 18:47:45
            print(f'days: {self.trips[" pickup_datetime"].apply(lambda x: arrow.get(str(x), "YYYY-MM-DD")).nunique():,}')   
        
        for col_lst in 'longitude latitude'.split():
                  self.trips[self.COLS[col_lst]] = self.trips[self.COLS[col_lst]] \
                                                          .applymap(lambda x: self.fix_coords(c=x, lat_lng=col_lst))
        
        for s in 'pickup dropoff'.split():
                  
            avail_mask = self.trips[f' {s}_longitude'].notnull() & self.trips[f' {s}_latitude'].notnull()
            self.trips[f'{s}_loc'] = '(' + self.trips[f' {s}_latitude'].astype(str) + ', ' + self.trips[f' {s}_longitude'].astype(str) + ')'
            self.trips[f'{s}_loc'] = self.trips[f'{s}_loc'].where(avail_mask, None)
                  
        self.trips['trip_'] = self.trips['pickup_loc'] + '->' + self.trips['dropoff_loc']
                  
        self.trips = self.trips.drop(self.COLS['latitude'] + self.COLS['longitude'], axis=1)
        
        return self
                  
    def get_passengers_per_trip(self):
        
        """
        returns a dictionary maps the number of passengeers to trip counts    
        """
                  
        # how do we define a trip in this case? "a cab with numberplate XXXX picked someone up at time TTTT" - that's a trip
        # this defines a trip in a unique way if we reasonably sssume that a specific cab can't do multiple pickupos at
        # the same time - if we do see that happening it's probably due to an error in data
                  
        TRIP_ID = ['medallion', ' pickup_datetime']
                  
        self.ANSWERS['passengers_per_trip'] = Counter(self.trips[TRIP_ID + [' passenger_count']] \
                                                              .drop_duplicates(TRIP_ID) \
                                                              .groupby(TRIP_ID) \
                                                              .sum()[' passenger_count'])
        return self
                  
    def get_payment_types(self):
        
        """
        returns a dictionary maps payment types to trip counts    
        """
                  
        TRIP_ID = ['medallion', ' pickup_datetime']
                  
        self.ANSWERS['payment_types'] = Counter(self.fares.drop_duplicates(TRIP_ID)[' payment_type'] \
                                                              .tolist())
        return self
                  
    def get_busiest_hours(self, hrs=5):
                  
        """
        return the top hrs busiest hours
        """
                  
        # so what is "busiest"? let's say it's when we see the most pickups or dropoffs; we couls just stick to pickups as that
        # would reflect both the drivers and clients perspective. However, dropoffs could matter to non-taxi drivers (traffic jams)
        # hence we get the busiest hours 
        
        self.ANSWERS['busiest_hrs'] = Counter(self.trips[' pickup_datetime'].dropna().apply(lambda x: arrow.get(x).hour).tolist() + \
                                                      self.trips[' dropoff_datetime'].dropna().apply(lambda x: arrow.get(x).hour).tolist()).most_common(hrs)
        return self
                  
    def get_busiest_locations(self, n=5):
                  
        """
        return the top n busiest location
        """
                  
        # busiest locations will be those where there are most pickups and/or dropoffs 

        self.ANSWERS['busiest_locations'] = sorted([(k,v)for k,v in {**Counter(self.trips['pickup_loc'].dropna()), 
                                             **Counter(self.trips['dropoff_loc'].dropna())}.items()], key=lambda x: x[1], reverse=True)[:n]
        return self
                  
    def get_most_unpredictable_trip(self):
                  
        """
        return a trip with the most varying travel time
        """
        ss = self.trips[['trip_', ' trip_time_in_secs']].dropna(subset=['trip_'])
        ss[' trip_time_in_secs'] = ss[' trip_time_in_secs'].astype(float)
                  
        ind = ss.groupby(['trip_']).std(ddof=0).idxmax().tolist()[0]
                  
        self.ANSWERS['most_unpredictable_trip'] = self.trips[self.trips['trip_'] == ind]
                  
        return self
        

    def get_trip_stats(self, rows_at_once=50000):
        
        """
        calculates 
        
            - the number of passengers per trip
            - top 5 busiest hours of the day
            - top 10 busiest locations
            - trip with the highest std of travel time
            
        these can be used to answer questions 
        
        1. What is the distribution of number of passengers per trip? (trips)
        6. What are top 5 busiest hours of the day? (trips/fares)
        7. What are the top 10 busiest locations of the city? (trips)
        8. Which trip has the highest standard deviation of travel time? (trips)
        """
        
        # trip is defined by (medallion, pickup_datetime)
        
        useful_columns = ['medallion', ' pickup_datetime', ' passenger_count', 
                          ' pickup_longitude', ' pickup_latitude', ' dropoff_longitude', 
                          ' dropoff_latitude', ' trip_time_in_secs']
        
        travel_times_sec = list()
        
        for i, d in enumerate(pd.read_csv('data/' + self.trip_file, chunksize=rows_at_once, 
                                          usecols=useful_columns)):
            for row in d.groupby(['medallion', ' pickup_datetime']).sum().iterrows():
                self.distros['passengers'][row[0]] += row[1][' passenger_count']
                self.distros['pickup_hrs'][arrow.get(row[0][1]).hour] += 1
                
                self.distros['loc'][self._round_coords((row[1][' pickup_latitude'], row[1][' pickup_longitude']))] += 1
                self.distros['loc'][(round(row[1][' dropoff_latitude'],3), round(row[1][' dropoff_longitude'],3))] += 1
                
                travel_times_sec.append(row[1][' trip_time_in_secs'])
                
            if i and (i%10 == 0):
                print(f'done {i*rows_at_once:,} rows..')
                
        self.STD_TRIP_TIME = np.std(travel_times_sec)
        self.AV_TRIP_TIME = np.mean(travel_times_sec)
        
        self.extreme_trip_index = travel_times_sec.index(max(travel_times_sec, key=lambda x: abs(x - self.AV_TRIP_TIME)))
        
        self.popular_locations = sorted([(','.join([str(k[0]), str(k[1])]), v) 
                                   for k, v in self.distros['loc'].items()], key=lambda x: x[1], reverse=True)
        
                
        return self
    
    def coords_to_street(self, coord):
                  
        """
        returns a street level description given location coordinates
        """
        coord = re.sub(r'[\(\)\[\]\s]+','', str(coord))
        
        try:
            res = nom.reverse(coord)
        except:
            print(f'can\'t figure out address for location ({coord})!')
            return None
                  
        if res.raw['address'].get('road', None):
            return res.raw['address']['road']
        else:
            return res.raw['address']
        
    
    def get_fare_stats(self, rows_at_once=50000):
        
        """
        2. What is the distribution of payment_type? (fares)
        3. What is the distribution of fare amount? (fares)
        4. What is the distribution of tip amount? (fares)
        5. What is the distribution of total amount? (fares)
        9. Which trip has most consistent fares? (fares)
        """
        
        # we don't need all columns to answer the questions, just these:
        
        self.fares = pd.read_csv('data/' + self.fare_file, usecols=self.trip_def_cols + self.amount_cols + self.payment_type_cols)
                  
        self.fare_distros = self.fares[self.trip_def_cols + self.amount_cols] \
                                    .groupby(self.trip_def_cols) \
                                    .sum() \
                                    .reset_index(drop=True)
        
        return self
    
    def get_most_consistent_trip(self):
                  
        """
        which trip has 'most consistent fares'? Firstly, what is a trip here? - it's a pair of locations (pickup -> dropoff);
        for each of such trips we'd like to find ot how much the fares vary and then select the trip with the least varying fare
        """
        
        g = self.trips[['medallion', ' pickup_datetime', 'trip_']] \
                  .join(self.fares[['medallion', ' pickup_datetime', ' total_amount']] \
                        .set_index(['medallion', ' pickup_datetime']),
                                        on=['medallion', ' pickup_datetime'], how='inner').reset_index()[['trip_', ' total_amount']]
        
        print(f'unique trips with fares: {g["trip_"].nunique():,}')
        
        g[' total_amount'] = g[' total_amount'].astype(float)
        
        # get the counts of trips; it makes more sense to look into fare variability for the trips done more than once
        cs = g.groupby(['trip_']).count()
                  
        tps = set(cs[cs[' total_amount'] > 1].reset_index()['trip_'])
        
        print(f'trips done more than once: {len(tps):,}')
        
        fare_stds = g[g['trip_'].isin(tps)].groupby('trip_').std()
        t = fare_stds[' total_amount'].idxmin()
        t_fares = g[g['trip_'] == t][' total_amount'].tolist()
              
        self.ANSWERS['trip_w_most_consistent_fares'] = {'trip': {'coords': t,
                                                                 'from_to': '->'.join(['(' + self.coords_to_street(loc) + ')' 
                                                                                       for loc in t.split('->')])},
                                                        'fares': t_fares}
              
        return self
        
        
        

In [3]:
if __name__ == '__main__':
    
    t0 = time.time()
    tx = Taxi(trip_file='trip_data_4.csv', fare_file='trip_fare_4.csv') \
                        .get_and_process_data(sample_n=6000).get_most_consistent_trip()
#     \
#     .get_payment_types() \
#     .get_most_consistent_trip()
#     .get_most_unpredictable_trip() \
    
#                             .get_busiest_hours() \
#                             .get_busiest_locations() \
#                             .get_passengers_per_trip()
    t1 = time.time()
    m, s = divmod(t1-t0, 60)
    print(f'elapsed time: {m:.0f} m {s:.0f} s')

unique trips with fares: 5,861
trips done more than once: 14
elapsed time: 1 m 23 s


In [11]:
tx.ANSWERS

defaultdict(None,
            {'trip_w_most_consistent_fares': {'trip': {'coords': '(40.769, -73.966)->(40.748, -73.986)',
               'from_to': '(Park Avenue)->(West 32nd Street)'},
              'fares': [11.0, 11.0]}})

In [9]:
tx.trips.head()

,medallion,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_loc,dropoff_loc,trip_
13176283,2CC2417152E8F5A4FD4D2CED69F8770E,2013-04-02 14:53:00,2013-04-02 15:07:00,1,840,1.36,"(40.751, -73.983)","(40.761, -73.983)","(40.751, -73.983)->(40.761, -73.983)"
4834620,8F30FFCCF85BA59647CBB745789E2EB4,2013-04-26 17:23:00,2013-04-26 17:30:00,2,420,.97,"(40.781, -73.955)","(40.77, -73.958)","(40.781, -73.955)->(40.77, -73.958)"
14504742,F9F739CDFADFB72EF5AB5FC7FBEEA027,2013-04-08 01:16:00,2013-04-08 01:24:00,6,480,2.21,"(40.779, -73.978)","(40.8, -73.961)","(40.779, -73.978)->(40.8, -73.961)"
12247963,B46EF45B234D91D4EA1ADACA6B560BCC,2013-04-11 12:27:42,2013-04-11 12:32:54,1,312,1.20,"(40.787, -73.948)","(40.795, -73.933)","(40.787, -73.948)->(40.795, -73.933)"
8578228,4814BAB4D79FBDA2ED80C538D933B0BA,2013-04-19 13:51:00,2013-04-19 14:10:00,6,1140,6.42,"(40.702, -74.013)","(40.763, -73.966)","(40.702, -74.013)->(40.763, -73.966)"


In [10]:
sns.set(rc={'figure.figsize':(12,6)})

sns.barplot(data=pd.DataFrame.from_dict(tx.ANSWERS['passengers_per_trip'], orient='index').reset_index().rename(columns={'index':'passengers', 0: 'count'}), x='passengers', y='count')

KeyError: 'passengers_per_trip'

In [ ]:
sns.barplot(data=pd.DataFrame.from_records(tx.ANSWERS['busiest_hrs']) \
            .rename(columns={0: 'hour', 1: 'count'}), x='hour', y='count')

In [ ]:
df = gpd.read_file(gpd.datasets.get_path('nybb'))

In [ ]:
df.head()

In [ ]:
ax = df.plot(figsize=(10, 10), alpha=0.5, edgecolor='k')